# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)
df.head()

In [ ]:
df.info()

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [ ]:
df.columns = df.columns.str.replace(" ", "_").str.replace("-", "_").str.lower()
list(df.columns)

In [ ]:
df_low_claim_yes = df[['total_claim_amount', 'response']]
df_low_claim_yes = df_low_claim_yes[(df_low_claim_yes['total_claim_amount'] < 1000) & (df_low_claim_yes['response'] == "Yes")]
df_low_claim_yes

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [ ]:
list(df.columns)

(1466, 26)

In [ ]:
df_responded_yes = df[df['response'] == "Yes"]
df_responded_yes['gender'].unique()

Yes    1466
Name: response, dtype: int64

In [ ]:
df_responded_yes.groupby(['policy_type', 'gender']).size()

In [ ]:
clv_mean = (
    df_responded_yes
        .groupby(['policy_type', 'gender'])['customer_lifetime_value']
        .mean()
        .reset_index()
)
clv_mean

In [ ]:
df.filter(like="premium").columns

In [ ]:
df_responded_yes = df[df["response"] == "Yes"].copy()

seg_summary = (
    df_responded_yes
    .groupby(["policy_type", "gender"])
    .agg(
        n=("customer", "size"),
        avg_premium=("monthly_premium_auto", "mean"),
        avg_clv=("customer_lifetime_value", "mean"),
        mean_claim=("total_claim_amount", "mean"),
        median_claim=("total_claim_amount", "median"),
        max_claim=("total_claim_amount", "max"),
    )
    .reset_index()
)

seg_summary


In [ ]:
seg_summary.sort_values("avg_clv", ascending=False)         # most profitable (CLV)

In [ ]:
seg_summary.sort_values("median_claim", ascending=True)     # lowest risk (claims)

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [ ]:
len(df)

In [ ]:
df["customer"].nunique()

In [ ]:
customers_by_state = df.groupby("state")["customer"].nunique()
customers_by_state

In [ ]:
customers_by_state_500 = customers_by_state[customers_by_state > 500].sort_values(ascending=False)
customers_by_state_500 

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [ ]:
clv_stats = (
    df.groupby(["education", "gender"])
        .agg(
            min_clv = ("customer_lifetime_value", "min"),
            median_clv = ("customer_lifetime_value", "median"),
            max_clv = ("customer_lifetime_value", "max"),
        )
        .reset_index()
)
clv_stats = clv_stats.round()
clv_stats

In [ ]:
clv_stats.sort_values(by="median_clv", ascending =False)

In [ ]:
clv_stats["max_minus_median"] = clv_stats["max_clv"] - clv_stats["median_clv"]
clv_stats.sort_values(by="max_minus_median", ascending=False)

In [ ]:
clv_stats["median_minus_min"] = clv_stats["median_clv"] - clv_stats["min_clv"]
clv_stats.sort_values(by="median_minus_min", ascending=False)

In [ ]:
## Highest median:
##    High school or below    M       6287

##    Doctor                  F       5332
## Biggest difference between median and max:
##    High school or below    M       77038
## Smallest difference between median and max:
##    Doctor                  M       27099
##Biggest difference between median and min:
##    High school or below    M       4346
## Smallest difference between median and min:
##    Doctor                  F       2936

In [ ]:
## Conclusion:
##- Most steady group is: Doctor F
##    - Doctor F has the smallest lower-tail spread and the second-smallest upper-tail spread, so overall it’s the most stable.
##- The 4 most steady groups are Doctor F & M, Master F & M.
##- Most unsteady group is: High school or below M.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [ ]:
list(df.columns)

In [ ]:
# rows: "state"
# columns: "effective_to_date"
# values: "number_of_policies"

df["effective_to_date"].dtype

In [ ]:
df["effective_to_date"].head()

In [ ]:
df["effective_to_date"] = pd.to_datetime(
    df["effective_to_date"],
    format="%m/%d/%y",
    errors="coerce"
)

df["effective_to_date"].dtype
df["effective_to_date"].isna().sum()

In [ ]:
df["month"] = df["effective_to_date"].dt.to_period("M")
df["month"].head()

In [ ]:
policies_state_month = pd.pivot_table(
    df,
    index = "state",
    columns = "month",
    values = "number_of_policies",
    aggfunc = "size",
    fill_value = 0
).sort_index()

policies_state_month

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [ ]:
policies_state_month.sort_values(by="2011-01", ascending=False)

In [ ]:
policies_state_month.sort_values(by=["2011-02", "2011-01"], ascending=False).head(3)

In [ ]:
top3_states = policies_state_month.sum(axis=1).sort_values(ascending=False).head(3).index
top3 = policies_state_month.loc[top3_states]
top3

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
df.filter(like="channel").columns

In [ ]:
df["responded_yes"] = (df["response"] == "Yes")

channel_response = (
    df.groupby("sales_channel")
      .agg(
          n=("customer", "size"),
          yes=("responded_yes", "sum"),
          response_rate=("responded_yes", "mean"),
      )
      .sort_values("response_rate", ascending=False)
      .reset_index()
)

channel_response